In [ ]:
!pip install transformers==3

     |████████████████████████████████| 757kB 5.6MB/s 
     |████████████████████████████████| 1.2MB 9.1MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 3.0MB 18.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e0d359fdc74d0d30c530868a5e42a41f513b02bc431da589b6ef601c68250b1c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


No GPU available, using the CPU instead.


In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
#model.cuda()
def answer_question(question, answer_text):

  # Tokenize
  input_ids = tokenizer.encode(question, answer_text)

  # Length of input sequence
  print('Query has {:,} tokens.\n'.format(len(input_ids)))

  # Set segment IDs
  sep_index = input_ids.index(tokenizer.sep_token_id)
  
  num_seg_a = sep_index + 1

  num_seg_b = len(input_ids) - num_seg_a

  segment_ids = [0]*num_seg_a + [1]*num_seg_b

  assert len(segment_ids) == len(input_ids)

  # Evaluate
  start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids = torch.tensor([segment_ids]))
  #input = input.to(device)
  #model = model.to(device)
  
  # Reconstruct answer
  # Find the tokens with the highest `start` and `end` scores.
  answer_start = torch.argmax(start_scores)
  answer_end = torch.argmax(end_scores)

  # Get the string versions of the input tokens.
  tokens = tokenizer.convert_ids_to_tokens(input_ids)

  # Start with the first token.
  answer = tokens[answer_start]

  # Select the remaining answer tokens and join them with whitespace.
  for i in range(answer_start + 1, answer_end + 1):
        
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

  print('Answer: "' + answer + '"')
  print(answer_start)

In [ ]:
cf_text = "Carbon fibers or carbon fibres (alternatively CF, graphite fiber or graphite fibre) are fibers about 5 to 10 micrometers (0.00020–0.00039 in) in diameter and composed mostly of carbon atoms. Carbon fibers have several advantages including high stiffness, high tensile strength, low weight, high chemical resistance, high temperature tolerance and low thermal expansion. These properties have made carbon fiber very popular in aerospace, civil engineering, military, and motorsports, along with other competition sports. However, they are relatively expensive when compared with similar fibers, such as glass fibers or plastic fibers. To produce a carbon fiber, the carbon atoms are bonded together in crystals that are more or less aligned parallel to the long axis of the fiber as the crystal alignment gives the fiber high strength-to-volume ratio (in other words, it is strong for its size). Several thousand carbon fibers are bundled together to form a tow, which may be used by itself or woven into a fabric. Carbon fibers are usually combined with other materials to form a composite. When permeated with a plastic resin and baked, it forms carbon-fiber-reinforced polymer (often referred to as carbon fiber) which has a very high strength-to-weight ratio, and is extremely rigid although somewhat brittle. Carbon fibers are also composited with other materials, such as graphite, to form reinforced carbon-carbon composites, which have a very high heat tolerance."

In [ ]:
question = ""

#question = question.to(device)

answer_question(question, cf_text)


Query has 297 tokens.

Answer: "."
